# GPU Subgridding Example
This notebook will explain how to use the GPU subgridding API to do FDTD Calculations. 

We'll start off with importing the Libraries.

In [ ]:
from pathlib import Path
import gprMax

We will get the file path step and 

In [ ]:
basefile = Path().resolve().parents[0]
fn = basefile/'sub-gridding'/'subgrid_basic.py'

## Defining the parameters
This section will define the required parameters that make up the model.

In [ ]:
# Discretisation is 1 mm in all 3 directions.
dl = 1e-3
# Sub-grid Ratio
ratio = 3
dl_s = dl / ratio

# Defining the pml parameters
pml_cells = 10
pml_gap = 15

# The distance between Inner Surface and Outer Surface of the subgridding
is_os_gap = 4
sub_gridded_region = 3

# Defining the domain size
extent = sub_gridded_region + 2 * (pml_cells + pml_gap + is_os_gap)

# Defining the domain extent
x = dl * extent
y = x
z = x

# Time window
tw = 2e-9

## Creating the scene
 
Here we will be creating the scene using the gprMax API.

In [ ]:
scene = gprMax.Scene()

title_gpr = gprMax.Title(name=fn.name)
dxdydz = gprMax.Discretisation(p1=(dl, dl, dl))
domain = gprMax.Domain(p1=(x, y, z))
time_window = gprMax.TimeWindow(time=tw)

scene.add(domain)
scene.add(title_gpr)
scene.add(dxdydz)
scene.add(time_window)

sg_x0 = (pml_cells + pml_gap + is_os_gap) * dl
sg_y0 = sg_x0
sg_z0 = sg_x0

sg_x1 = sg_x0 + sub_gridded_region * dl
sg_y1 = sg_x1
sg_z1 = sg_x1

sg_p0 = [sg_x0, sg_y0, sg_z0]
sg_p1 = [sg_x1, sg_y1, sg_z1]

Now we create the subgrid and initialise it for GPU by passing `gpu=True`.

In [ ]:
sg = gprMax.SubGridHSG(p1=sg_p0, p2=sg_p1, ratio=ratio, id='mysubgrid', gpu=True)
scene.add(sg)

Adding a waveform, source and receiver. The source will be inside the subgrid while the receiver outside the subgrid.

In [ ]:
wave = gprMax.Waveform(wave_type = 'gaussiandotnorm', amp = 1, freq = 1e9, id = 'my_waveform')
sg.add(wave)

source = gprMax.HertzianDipole(polarisation = 'z', p1=(30*dl, 30*dl, 30*dl), waveform_id = 'my_waveform')
sg.add(source)

rec = gprMax.Rx(p1 = (25*dl, 30*dl, 30*dl))
scene.add(rec)

Next line of code creates teh geometry view of the sub grid only. The command currently exports all the entire subgrid regardless of `p1` and `p2`.

In [ ]:
gv_sg_normal = gprMax.GeometryView(p1=sg_p0,
                         p2=sg_p1,
                         dl=(1e-3, 1e-3, 1e-3),
                         filename=fn.with_suffix('').parts[-1] + '_subgrid_normal',
                         output_type='n')

# add the subgrid geometry view to the sub grid object 
sg.add(gv_sg_normal)

## Running the simulation
We use the `gprMax.run()` command to run the simulation with the tag `gpu='gpu'`.

In [ ]:
gprMax.run(scenes=[scene], n=3, geometry_only=False, outputfile=fn, subgrid=True, gpu='gpu', autotranslate=True)